In [5]:
import datetime
from dateutil.relativedelta import relativedelta

from marcap import marcap_data
import pandas as pd

def get_data(day):
  fav_stocks = {
    '005930': '삼성전자', 
    '005380': '현대차', 
    '015760': '한국전력', 
    '005490': 'POSCO', 
    '105560': 'KB금융',
  }
  
  year_from = day - relativedelta(years=1) - relativedelta(day=1)
  year_to = day - relativedelta(day=1)
  df = marcap_data(year_from, day)
  # df= df[df['Code'].isin(fav_stocks.keys())]
  
  today_df = df.loc[str(day)]

  df = df.loc[year_from : year_to]
  df = df[['Code', 'Name', 'Close']]
  gg = df.groupby(['Code'])
  first_df = gg.first().rename(columns={'Close':'1-YrFirst'})
  max_df = gg.max().rename(columns={'Close':'1-YrHigh'})
  min_df = gg.min().rename(columns={'Close':'1-YrLow'})
  
  # merge aggregated values min/max/first
  new_df = today_df
  new_df = pd.merge(new_df, first_df)
  new_df = pd.merge(new_df, min_df)
  new_df = pd.merge(new_df, max_df)
  
  new_df['Result'] = False
  for index, co in new_df.iterrows():
    if co['1-YrFirst'] >= co['1-YrHigh']:
      if co['1-YrFirst'] < co['Close']:
        new_df.at[index,'Result'] = True
  
  mask = new_df.Result == True
  return new_df.loc[mask,:]



In [7]:
today = datetime.date.today()
day = today - relativedelta(days=1)
df = get_data(day)
df

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,...,Volume,Amount,Marcap,Stocks,MarketId,Rank,1-YrFirst,1-YrLow,1-YrHigh,Result
287,192400,쿠쿠홀딩스,KOSPI,NaN,109500,1,2500,2.34,107500,110000,...,23867,2562898500,778811851500,7112437,STK,298,107000,65800,107000,True


In [9]:
day = day - relativedelta(days=1)
df = get_data(day)
df

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,...,Volume,Amount,Marcap,Stocks,MarketId,Rank,1-YrFirst,1-YrLow,1-YrHigh,Result
